# imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import networkx as nx
import sklearn
import xgboost as xgb
import pickle 

# sklearn
from sklearn import model_selection # split함수이용
from sklearn import ensemble # RF,GBM
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

# gnn
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv

# autogluon
from autogluon.tabular import TabularDataset, TabularPredictor

/home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/torch_geometric/typing.py:18: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/libpyg.so: undefined symbol: _ZN2at4_ops12split_Tensor4callERKNS_6TensorEN3c106SymIntEl
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/torch_geometric/typing.py:31: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/torch_scatter/_scatter_cuda.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/torch_geometric/typing.py:42: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /home/coco/anaconda3/en

In [2]:
def throw(df, fraud_rate):  # 사기 거래 비율에 맞춰 버려지는 함수!
    df1 = df[df['is_fraud'] == 1].copy()
    df0 = df[df['is_fraud'] == 0].copy()
    df0_downsample = (len(df1) * (1-fraud_rate)) / (len(df0) * fraud_rate)
    df0_down = df0.sample(frac=df0_downsample, random_state=42)
    df_p = pd.concat([df1, df0_down])
    return df_p

def split_dataframe(data_frame, test_fraud_rate, test_rate=0.3):
    n = len(data_frame)

    # 사기 거래와 정상 거래를 분리
    fraud_data = data_frame[data_frame['is_fraud'] == 1]
    normal_data = data_frame[data_frame['is_fraud'] == 0]

    # 테스트 데이터 크기 계산
    test_samples = int(test_fraud_rate * (n * test_rate))
    remaining_test_samples = int(n * test_rate) - test_samples

    # 사기 거래 및 정상 거래에서 무작위로 테스트 데이터 추출
    test_fraud_data = fraud_data.sample(n=test_samples, replace=False)
    test_normal_data = normal_data.sample(n=remaining_test_samples, replace=False)

    # 테스트 데이터 합치기
    test_data = pd.concat([test_normal_data, test_fraud_data])

    # 훈련 데이터 생성
    train_data = data_frame[~data_frame.index.isin(test_data.index)]

    return train_data, test_data

def concat(df_tr, df_tst):   
    df = pd.concat([df_tr, df_tst])
    train_mask = np.concatenate((np.full(len(df_tr), True), np.full(len(df_tst), False)))    # index꼬이는거 방지하기 위해서? ★ (이거,, 훔,,?(
    test_mask =  np.concatenate((np.full(len(df_tr), False), np.full(len(df_tst), True))) 
    mask = (train_mask, test_mask)
    return df, mask

def evaluation(y, yhat):
    metrics = [sklearn.metrics.accuracy_score,
               sklearn.metrics.precision_score,
               sklearn.metrics.recall_score,
               sklearn.metrics.f1_score,
               sklearn.metrics.roc_auc_score]
    return pd.DataFrame({m.__name__:[m(y,yhat).round(6)] for m in metrics})

def compute_time_difference(group):
    n = len(group)
    result = []
    for i in range(n):
        for j in range(n):
            time_difference = abs((group.iloc[i].trans_date_trans_time - group.iloc[j].trans_date_trans_time).total_seconds())
            result.append([group.iloc[i].name, group.iloc[j].name, time_difference])
    return result

def edge_index_save(df, unique_col, theta, gamma):
    groups = df.groupby(unique_col)
    edge_index = np.array([item for sublist in (compute_time_difference(group) for _, group in groups) for item in sublist])
    edge_index = edge_index.astype(np.float64)
    filename = f"edge_index_attempt{self.save_attempt}_{str(unique_col).replace(' ', '').replace('_', '')}.npy"

    while os.path.exists(filename):
        self.save_attempt += 1
        filename = f"edge_index_attempt{self.save_attempt}_{str(unique_col).replace(' ', '').replace('_', '')}.npy"
    np.save(filename, edge_index)
    #tetha = edge_index_plust_itme[:,].mean()


    edge_index[:,2] = (np.exp(-edge_index[:,2]/(theta)) != 1)*(np.exp(-edge_index[:,2]/(theta))).tolist()
    edge_index = torch.tensor([(int(row[0]), int(row[1])) for row in edge_index if row[2] > gamma], dtype=torch.long).t()
    return edge_index

def edge_index(df, unique_col, theta, gamma):
    groups = df.groupby(unique_col)
    edge_index = np.array([item for sublist in (compute_time_difference(group) for _, group in groups) for item in sublist])
    edge_index = edge_index.astype(np.float64)
   # filename = f"edge_index_attempt{self.save_attempt}_{str(unique_col).replace(' ', '').replace('_', '')}.npy"

    # while os.path.exists(filename):
    #     self.save_attempt += 1
    #     filename = f"edge_index_attempt{self.save_attempt}_{str(unique_col).replace(' ', '').replace('_', '')}.npy"
    # np.save(filename, edge_index)
    #tetha = edge_index_plust_itme[:,].mean()


    edge_index[:,2] = (np.exp(-edge_index[:,2]/(theta)) != 1)*(np.exp(-edge_index[:,2]/(theta))).tolist()
    edge_index = torch.tensor([(int(row[0]), int(row[1])) for row in edge_index if row[2] > gamma], dtype=torch.long).t()
    return edge_index

In [3]:
with open('../fraudTrain.pkl', 'rb') as file:
    fraudTrain = pickle.load(file)    

# Autogluon(df02)

In [4]:
fraudTrain = fraudTrain[["amt","is_fraud"]]

In [5]:
# def auto(df,test_fraud_rate):
#     df_tr, df_tst = split_dataframe(df, test_fraud_rate)
#     tr = TabularDataset(df_tr)
#     tst = TabularDataset(df_tst)
#     predictr = TabularPredictor("is_fraud")
#     predictr.fit(tr, presets='best_quality')
#     y = tst.is_fraud
#     yhat = predictr.predict(tst)
#     result = evaluation(y,yhat)
#     return result

In [6]:
df = throw(fraudTrain, 0.3)
df_tr, df_tst = split_dataframe(df, 0.5)
tr = TabularDataset(df_tr)
tst = TabularDataset(df_tst)
predictr = TabularPredictor("is_fraud")
predictr.fit(tr, presets='best_quality')
y = tst.is_fraud
yhat = predictr.predict(tst)
result1 = evaluation(y,yhat)

No path specified. Models will be saved in: "AutogluonModels/ag-20240202_071009/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240202_071009/"
AutoGluon Version:  0.8.2
Python Version:     3.8.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #38~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov  2 18:01:13 UTC 2
Disk Space Avail:   590.08 GB / 982.82 GB (60.0%)
Train Data Rows:    14014
Train Data Columns: 1
Label Column: is_fraud
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> 

In [7]:
predictr.evaluate(tst)

Evaluation: accuracy on test data: 0.8519813519813519
Evaluations on test data:
{
    "accuracy": 0.8519813519813519,
    "balanced_accuracy": 0.8519813519813519,
    "mcc": 0.7293218533016024,
    "roc_auc": 0.9617966670913722,
    "f1": 0.8297261061099407,
    "precision": 0.9765554553651938,
    "recall": 0.7212787212787213
}


{'accuracy': 0.8519813519813519,
 'balanced_accuracy': 0.8519813519813519,
 'mcc': 0.7293218533016024,
 'roc_auc': 0.9617966670913722,
 'f1': 0.8297261061099407,
 'precision': 0.9765554553651938,
 'recall': 0.7212787212787213}

In [8]:
predictr._trainer.model_graph.nodes

NodeView(('KNeighborsUnif_BAG_L1', 'KNeighborsDist_BAG_L1', 'LightGBMXT_BAG_L1', 'LightGBM_BAG_L1', 'RandomForestGini_BAG_L1', 'RandomForestEntr_BAG_L1', 'CatBoost_BAG_L1', 'ExtraTreesGini_BAG_L1', 'ExtraTreesEntr_BAG_L1', 'NeuralNetFastAI_BAG_L1', 'XGBoost_BAG_L1', 'NeuralNetTorch_BAG_L1', 'LightGBMLarge_BAG_L1', 'WeightedEnsemble_L2'))

In [10]:
predictr.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      LightGBMLarge_BAG_L1   0.926145       0.023825   1.000465                0.023825           1.000465            1       True         13
1           LightGBM_BAG_L1   0.926145       0.030142   0.756562                0.030142           0.756562            1       True          4
2       WeightedEnsemble_L2   0.926145       0.045217   3.698785                0.015075           2.942223            2       True         14
3     NeuralNetTorch_BAG_L1   0.925931       0.068089  13.514495                0.068089          13.514495            1       True         12
4    NeuralNetFastAI_BAG_L1   0.925860       0.144026  12.193488                0.144026          12.193488            1       True         10
5           CatBoost_BAG_L1   0.924718       0.005455   1.765058                

/home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/autogluon/core/utils/plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'RandomForestGini_BAG_L1': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesGini_BAG_L1': 'StackerEnsembleModel_XT',
  'ExtraTreesEntr_BAG_L1': 'StackerEnsembleModel_XT',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'NeuralNetTorch_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel'},
 'model_performance': {'KNeighborsUnif_BAG_L1': 0.9170115598687028,
  'KNeighborsDist_BAG_L1': 0.9083773369487655,
  'LightGBMXT_BAG_L1': 0.9233623519337805,
  'LightGBM_BAG_L1': 0.9261452832881404,
  'R

In [11]:
predictr.get_model_names()

['KNeighborsUnif_BAG_L1',
 'KNeighborsDist_BAG_L1',
 'LightGBMXT_BAG_L1',
 'LightGBM_BAG_L1',
 'RandomForestGini_BAG_L1',
 'RandomForestEntr_BAG_L1',
 'CatBoost_BAG_L1',
 'ExtraTreesGini_BAG_L1',
 'ExtraTreesEntr_BAG_L1',
 'NeuralNetFastAI_BAG_L1',
 'XGBoost_BAG_L1',
 'NeuralNetTorch_BAG_L1',
 'LightGBMLarge_BAG_L1',
 'WeightedEnsemble_L2']

In [21]:
models = predictr._trainer.model_graph.nodes

# 결과를 저장할 데이터프레임 생성
results = pd.DataFrame(columns=['Model', 'Evaluation'])

# 각 모델에 대해 평가 수행 및 결과 저장
for model_name in models:
    # 모델 평가
    eval_result = predictr.evaluate(tst, model=model_name)
    
    # 결과를 데이터프레임에 추가
    results = results.append({'Model': model_name, 'Evaluation': eval_result}, ignore_index=True)

Evaluation: accuracy on test data: 0.8591408591408591
Evaluations on test data:
{
    "accuracy": 0.8591408591408591,
    "balanced_accuracy": 0.8591408591408591,
    "mcc": 0.7333692346586707,
    "roc_auc": 0.9238014178074118,
    "f1": 0.8433333333333334,
    "precision": 0.9499374217772215,
    "recall": 0.7582417582417582
}
/tmp/ipykernel_3624978/2085143255.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Model': model_name, 'Evaluation': eval_result}, ignore_index=True)
Evaluation: accuracy on test data: 0.8546453546453546
Evaluations on test data:
{
    "accuracy": 0.8546453546453546,
    "balanced_accuracy": 0.8546453546453547,
    "mcc": 0.7243419861043393,
    "roc_auc": 0.9035307017324999,
    "f1": 0.8382434685936633,
    "precision": 0.9448621553884712,
    "recall": 0.7532467532467533
}
/tmp/ipykernel_3624978/2085143255.py:12: FutureWarning: The frame.ap

In [22]:
results

,Model,Evaluation
0,KNeighborsUnif_BAG_L1,"{'accuracy': 0.8591408591408591, 'balanced_accuracy': 0.8591408591408591, 'mcc': 0.7333692346586707, 'roc_auc': 0.9238014178074118, 'f1': 0.8433333333333334, 'precision': 0.9499374217772215, 'recall': 0.7582417582417582}"
1,KNeighborsDist_BAG_L1,"{'accuracy': 0.8546453546453546, 'balanced_accuracy': 0.8546453546453547, 'mcc': 0.7243419861043393, 'roc_auc': 0.9035307017324999, 'f1': 0.8382434685936633, 'precision': 0.9448621553884712, 'recall': 0.7532467532467533}"
2,LightGBMXT_BAG_L1,"{'accuracy': 0.8576423576423576, 'balanced_accuracy': 0.8576423576423576, 'mcc': 0.7357286217082192, 'roc_auc': 0.9497830951377405, 'f1': 0.8387705072600414, 'precision': 0.9669565217391304, 'recall': 0.7405927405927406}"
3,LightGBM_BAG_L1,"{'accuracy': 0.8519813519813519, 'balanced_accuracy': 0.8519813519813519, 'mcc': 0.7293218533016024, 'roc_auc': 0.9617966670913722, 'f1': 0.8297261061099407, 'precision': 0.9765554553651938, 'recall': 0.7212787212787213}"
4,RandomForestGini_BAG_L1,"{'accuracy': 0.8463203463203464, 'balanced_accuracy': 0.8463203463203464, 'mcc': 0.7069430711855968, 'roc_auc': 0.9305546823029339, 'f1': 0.829232192414431, 'precision': 0.9329725228975854, 'recall': 0.7462537462537463}"
5,RandomForestEntr_BAG_L1,"{'accuracy': 0.8463203463203464, 'balanced_accuracy': 0.8463203463203464, 'mcc': 0.7069430711855968, 'roc_auc': 0.9305546823029339, 'f1': 0.829232192414431, 'precision': 0.9329725228975854, 'recall': 0.7462537462537463}"
6,CatBoost_BAG_L1,"{'accuracy': 0.8613053613053613, 'balanced_accuracy': 0.8613053613053614, 'mcc': 0.7411229848770046, 'roc_auc': 0.961435722674484, 'f1': 0.8439782730848474, 'precision': 0.9644691780821918, 'recall': 0.7502497502497503}"
7,ExtraTreesGini_BAG_L1,"{'accuracy': 0.8553113553113553, 'balanced_accuracy': 0.8553113553113554, 'mcc': 0.7239232471410014, 'roc_auc': 0.938045415567893, 'f1': 0.8400515369041046, 'precision': 0.9390946502057613, 'recall': 0.7599067599067599}"
8,ExtraTreesEntr_BAG_L1,"{'accuracy': 0.8546453546453546, 'balanced_accuracy': 0.8546453546453546, 'mcc': 0.7228538607711172, 'roc_auc': 0.9375860569666764, 'f1': 0.8391376451077943, 'precision': 0.9393564356435643, 'recall': 0.7582417582417582}"
9,NeuralNetFastAI_BAG_L1,"{'accuracy': 0.8614718614718615, 'balanced_accuracy': 0.8614718614718615, 'mcc': 0.740949374063438, 'roc_auc': 0.9046451938559831, 'f1': 0.8444278234854151, 'precision': 0.9628997867803838, 'recall': 0.7519147519147519}"


In [ ]:
results

[{'model': 'KNeighborsUnif_BAG_L1',
  'acc': 0.8541458541458542,
  'pre': 0.9411032766486935,
  'rec': 0.7555777555777555,
  'f1': 0.838197266346509,
  'auc': 0.9230023500752773},
 {'model': 'KNeighborsDist_BAG_L1',
  'acc': 0.8481518481518482,
  'pre': 0.933997509339975,
  'rec': 0.7492507492507493,
  'f1': 0.8314855875831486,
  'auc': 0.8990750064676138},
 {'model': 'LightGBMXT_BAG_L1',
  'acc': 0.8571428571428571,
  'pre': 0.9669133652590335,
  'rec': 0.7395937395937396,
  'f1': 0.8381132075471698,
  'auc': 0.9521417088849656},
 {'model': 'LightGBM_BAG_L1',
  'acc': 0.8486513486513486,
  'pre': 0.9746146872166818,
  'rec': 0.7159507159507159,
  'f1': 0.8254943367248992,
  'auc': 0.9625852003474381},
 {'model': 'RandomForestGini_BAG_L1',
  'acc': 0.8426573426573427,
  'pre': 0.925206611570248,
  'rec': 0.7455877455877455,
  'f1': 0.825742209109349,
  'auc': 0.9291861984169677},
 {'model': 'RandomForestEntr_BAG_L1',
  'acc': 0.8426573426573427,
  'pre': 0.925206611570248,
  'rec': 0.7